<a href="https://colab.research.google.com/github/AnitaKirkovska/MachineLearning_HW1/blob/master/Problem2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.special
from tqdm import tqdm_notebook as tqdm

The Network model


In [0]:
class neuralArch:
  
  def __init__ (self, inputnodes, outputnodes, learningrate):
    self.inodes = inputnodes
    self.onodes = outputnodes
    
    #link weight natrices. wio -> input and hidden, who -> hidden and output
    self.wio = np.random.normal(0.0, pow(self.inodes, -0.5), (self.onodes, self.inodes))
    #learning rate
    self.lr = learningrate
    
    #activation function -> sigmoid
    self.activation_function = lambda x: scipy.special.expit(x)
    
    pass
  
  def train(self, inputs_list, targets_list):
    
    #convert to 2d array
    inputs = np.array(inputs_list, ndmin=2).T
    targets = np.array(targets_list, ndmin=2).T
    
    #calculating signals into last layer 
    last_inputs = np.dot(self.wio,inputs)
    last_outputs = self.activation_function(last_inputs)
    
    #output layer error 
    last_errors = targets - last_outputs
    
    #update the weight links between input layers and last layers
    self.wio += self.lr * np.dot((last_errors * last_outputs * (1.0 - last_outputs)), np.transpose(inputs))
    
    pass
  
  def predict(w, b, X):
    m = X.shape[1]
    Y_pred = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute the prediction vector "A" containing the probabilities
    A = sigmoid(np.dot(w.T, X) + b)
       
    for i in range(A.shape[0]):
        # Convert probabilities a[0,i] to actual predictions p[0,i]
        if A[0][i] <= 0.5:
            Y_pred[0, i] = 0
        else:
            Y_pred[0, i] = 1
    return Y_pred
  
  def query(self,inputs_list):
    
    #convert to 2d array
    inputs = np.array(inputs_list, ndmin=2).T
    
    #calculating signals into and from last layer 
    last_inputs = np.dot(self.wio,inputs)
    last_outputs = self.activation_function(last_inputs)
    
    return last_outputs

Loading the data

In [4]:
from keras.datasets import mnist

(x_train_org, y_train_org), (x_test_org, y_test_org) = mnist.load_data()

Using TensorFlow backend.


Preparing the data (Range 1-0)

In [6]:
x_train = x_train_org.reshape((60000, 28 * 28))
scaled_input_x = np.asfarray(x_train)/255.0

x_test = x_test_org.reshape((10000, 28 * 28))
scaled_test_x = np.asfarray(x_test)/255.0


scaled_input_x
scaled_test_x


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Target matrix - the labels y

In [0]:
from keras.utils import to_categorical

y_train_org = to_categorical(y_train_org)
y_test_org = to_categorical(y_test_org)

Nodes, learning rate and instance of the network architecture (mini-batch)

In [0]:
#output nodes are 10, the # of classifiers we have
input_nodes = len(scaled_input_x[0])
output_nodes = len(y_train_org[0])

learning_rate = 0.3

#instance of the Network
n = neuralArch(input_nodes, output_nodes, learning_rate)

Training the network

In [0]:
epochs = 6
batch_size = 4

for epoch in tqdm(range(epochs)):
    shuffled_indices = np.random.permutation(len(scaled_input_x))
    X_b_shuffled = scaled_input_x[shuffled_indices]
    y_shuffled = y_train_org[shuffled_indices]
    
    
    
    for i in tqdm(range(0, len(scaled_input_x), batch_size)):
      xi = X_b_shuffled[i:i+batch_size]
      yi = y_shuffled[i:i+batch_size]
      n.train(xi, yi)

  

Testing 

In [0]:
answers = []
for i in tqdm(range(len(scaled_test_x))):
  answer = n.query(scaled_test_x[i])
  answers.append(answer.argmax())

In [0]:
performance= []
for i in answers:
  if answers[i] == y_test_org.argmax():
    performance.append(1)
  else:
    performance.append(0)
    pass 

In [0]:
performance_array = np.asarray(performance)
print ("performance = ", performance_array.sum() / performance_array.size)